<a href="https://colab.research.google.com/github/WTurunen/cookbook/blob/main/examples/Story_Writing_with_Prompt_Chaining-edit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2024 Google LLC.

In [100]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Prompt Chaining and Iterative Generation for Story Writing

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/examples/Story_Writing_with_Prompt_Chaining.ipynb"><img src="https://github.com/google-gemini/cookbook/blob/main/images/colab_logo_32px.png?raw=1" />Run in Google Colab</a>
  </td>
</table>

This notebook demonstrates how to write a story using two  powerful tools: prompt chaining and iterative generation. These can be used to tackle complex tasks that are difficult or impossible to complete in a single step.

**Prompt chaining** involves breaking down a larger task into smaller, interconnected prompts. The output of each prompt then becomes the input for the next, guiding the language model through the process step-by-step. This approach offers several benefits:

*   Improved accuracy: Smaller, focused prompts can lead to better results from the language model.
*   Debugging: It's easier to identify where things go wrong within the chain, allowing for targeted adjustments and improvements.
*   Complex tasks: By breaking down intricate problems into manageable steps, prompt chaining enables the language model to tackle more complex tasks.

**Iterative generation** refers to the process of building the desired output iteratively. In this case, you will use it to write a story that is longer than what a single generation window allows. Iterative generation offers several benefits:

*   Longer outputs: It allows for the creation of longer and more detailed outputs, exceeding the limitations of a single generation window.
*   Flexibility: You can adjust and refine the output at each iteration, ensuring the story develops in the desired direction.
*   Human-in-the-loop control: You can provide feedback and guidance at each step, ensuring the story aligns with your creative vision.

By combining these techniques, you can create a compelling and well-structured story, piece by piece, while maintaining control over the creative process.


## Setup

In [101]:
! pip install -q -U "google-generativeai>=0.7.2"

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see the [Authentication](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) quickstart for an example.

In [102]:
import google.generativeai as genai
from google.api_core import retry
from google.colab import userdata
from pprint import pprint

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

model = genai.GenerativeModel('gemini-2.0-flash-exp')

# For convenience, a simple wrapper to let the SDK handle error retries
def generate_with_retry(model, prompt):
  return model.generate_content(prompt, request_options={'retry':retry.Retry()})

## Prompts: Guiding the language model

You will use a series of interconnected prompts to guide the language model through the process of writing a story. These prompts will cover the story's premise, outline, and starting point, ultimately leading to the creation of a complete narrative.

It's important to carefully craft these prompts to provide clear instructions and relevant information to the language model. This will help the model generate high-quality content that aligns with your creative vision.


### Prompt chain for story writing

This section contains the prompts that will guide the language model through the story writing process. These prompts are designed to be chained together, with the output of one prompt feeding into the next.

Each prompt includes a **persona statement**, which helps the language model understand its role and generate more relevant and accurate content. In this case, the persona statement is: `You are an award-winning science fiction author with a penchant for expansive, intricately woven stories. Your ultimate goal is to write the next award winning sci-fi novel.`

Since the persona statement and writing guidelines appear in multiple prompts, f-string variables are used to add them to the prompts.

Additionally, the prompts use **placeholders** to insert the results of previous prompts using `.format()`. This allows us to build the story step-by-step, incorporating the outputs generated by the model at each stage. These are normally denoted by `{}`, but since you are also using f-string variables they are escaped as `{{}}`.

Here's a breakdown of the prompt chain:

1.  **Premise prompt**: This prompt asks the model to generate a single-sentence premise for a sci-fi story featuring cats.
1.  **Outline prompt**: This prompt provides the generated premise to the model and asks it to create a plot outline for the story.
1.  **Starting prompt**: This prompt provides both the premise and the outline to the model and asks it to begin writing the story. It also includes instructions to write a detailed and lengthy opening section.

By chaining these prompts together, it guides the language model through the process of creating a well-structured and engaging story.

#### Example

Let's say the model generates the following premise:

> In a future where humanity has achieved interstellar travel, a group of genetically enhanced cats embarks on a perilous mission to save the galaxy from a sinister alien threat.

This premise would then be inserted into the outline prompt:

> You are an award-winning science fiction author with a penchant for expansive,
intricately woven stories. Your ultimate goal is to write the next award winning
sci-fi novel.
>
> You have a gripping premise in mind:
>
> **In a future where humanity has achieved interstellar travel, a group of genetically enhanced cats embarks on a perilous mission to save the galaxy from a sinister alien threat.**
>
> Write an outline for the plot of your story.


The model would then generate an outline based on this premise, which would then be used in the starting prompt to begin writing the story itself.

This process continues iteratively, with the model generating additional content based on the previous outputs, until the story is complete.


In [103]:
persona = '''\
Olet lämminhenkinen ja mielikuvituksellinen lastenkirjailija, joka luo leikkisiä tarinoita.
täynnä ihmeitä ja seikkailuja. Perimmäinen tavoitteesi on kirjoittaa ihastuttavia
tarinoita, jotka sytyttävät nuoria mieliä ja saavat 6-vuotiaat kikattavat ilosta.
'''

guidelines = '''\
Kirjoitusohjeet

Tee maailmasta hauska ja mukaansatempaava. Luo värikkäitä, mielikuvituksellisia kohtauksia, jotka lapsi voi helposti kuvitella mielessään. Pidä hahmot yksinkertaisina mutta mieleenpainuvina - anna heille selkeät persoonallisuudet ja tunteet, joihin nuoret lukijat voivat samaistua. Seuraa tarinasi suuntaa, mutta anna uusien leikkisien ideoiden tulla esiin luonnollisesti kirjoittaessasi. Käytä kirkkaita, eloisia ja eri aisteihin vetoavia kuvauksia, jotta ympäristö ja hahmot tuntuvat todellisilta ja jännittäviltä. Ripottele hahmoista tai paikoista pieniä yksityiskohtia, jotka voivat johtaa myöhemmin hauskoihin sivuseikkailuihin. Pidä asiat viihdyttävinä vihjailemalla, mitä seuraavaksi voi tapahtua. Älä päätä kaikkea liian nopeasti - jätä tilaa uusille seikkailuille. Istuta pieniä tarinan siemeniä, jotka voivat kasvaa suuremmiksi yllätyksiksi tarinan jatkuessa.

Muista, että päätavoitteesi on pitää tarinan kulku luonnollisena. Jos kiirehdit tapahtumien läpi liian nopeasti, se ei ole hyvä asia. Ota aikaa hauskojen hetkien tutkimiseen, älä koskaan vain tiivistele niitä.

Kirjoita suomeksi.

'''

premise_prompt = f'''\
{persona}

Kirjoita yhden lauseen lähtökohta lastentarinalle, jossa esiintyy marsuja. Kirjoita suomeksi.'''

outline_prompt = f'''\
{persona}

Sinulla on mielessäsi mukaansatempaava lähtökohta:

{{premise}}

Kirjoita hahmotelma tarinasi juonesta. Kirjoita suomeksi.'''

starting_prompt = f'''\
{persona}

Sinulla on mielessäsi mukaansatempaava lähtökohta:

{{premise}}

Mielikuvituksesi on luonut rikkaan kerronnallisen hahmotelman:

{{outline}}

Tarkastele ensin hiljaisesti pääpiirteittäistä tekstiä ja lähtökohtia. Mieti, miten aloitat
tarina.

Aloittakaa tarinan alun kirjoittaminen. Sinun ei odoteta lopettavan
koko tarinaa nyt. Kirjoituksesi pitäisi olla niin yksityiskohtainen, että olet vain
raaputtelet pintaa hahmotelmasi ensimmäisestä kappaleesta. Yritä kirjoittaa AT
Vähintään 1000 sanaa ja enintään 2000 sanaa.

{guidelines}'''

### Continuation prompt: Building the story

Once the language model has generated the beginning of the story, you can use a **continuation prompt** to iteratively expand the narrative. This prompt is similar to the starting prompt, but with two key differences:

1.  **Instruction to signal completion**: An instruction was added for the model to write `IAMDONE` when it believes the story is finished. This serves as a signal for us to stop generating additional content.
1.  **Work in progress**: The language in the prompt is adjusted to reflect that the story is already in progress, rather than starting from scratch.

The continuation prompt provides the model with the story's premise, outline, and the existing draft. It then instructs the model to continue writing the story in detail.

This iterative process allows us to build a longer and more complex story than what would be possible in a single generation call. You can continue feeding the existing draft back into the continuation prompt until the model signals that the story is complete by writing `IAMDONE`.

**Note**: The `IAMDONE` signal is simply a convenient way to identify the story's end in this example. In other applications of iterative generation, different methods might be used to determine when the desired output is complete.


In [104]:
continuation_prompt = f'''\
{persona}

Sinulla on mielessäsi mukaansatempaava lähtökohta:

{{premise}}

Mielikuvituksesi on luonut rikkaan kerronnallisen hahmotelman:

{{outline}}

Olet alkanut uppoutua tähän maailmaan, ja sanat virtaavat.
Tässä on, mitä olet kirjoittanut tähän mennessä:

{{story_text}}

=====

Käy ensin hiljaa läpi tähänastinen hahmotelma ja tarina. Tunnistakaa, mitä yksittäinen
seuraava osa hahmotelmastasi, joka sinun pitäisi kirjoittaa.

Tehtävänäsi on jatkaa siitä, mihin jäit, ja kirjoittaa tarinan seuraava osa.
Sinun ei odoteta saavasi koko tarinaa nyt valmiiksi. Kirjoituksesi pitäisi olla
niin yksityiskohtaista, että raaputat vain pintaa seuraavasta osasta.
hahmotelmaa. Yritä kirjoittaa vähintään 1000 sanaa. Kuitenkin vasta sitten, kun tarina
on TÄYSIN valmis, kirjoita IAMDONE. Muista, että ÄLÄ kirjoita kokonaista lukua
juuri nyt.

{guidelines}'''

## Writing time!

### Generate and print the premise

In [105]:
premise = generate_with_retry(model, premise_prompt).text
print(premise)

Pörröinen marsu nimeltä Pomppu löytää salaisen oven perunakellarin takaa, ja se johtaa sateenkaaren värisiin tunneleihin täynnä nauravia porkkanoita.



### Generate and print the outline

In [106]:
outline = generate_with_retry(model, outline_prompt.format(premise=premise)).text
print(outline)

Okei, tästä tulee ihana tarina! Tässä on hahmotelma Pomppu-marsun seikkailulle sateenkaarimaisiin tunneleihin:

**Otsikko:** Pomppu ja Naurahtavat Porkkanatunnelit

**Päähenkilö:**

*   **Pomppu:** Pörröinen, utelias ja hieman rohkea marsu, jolla on tapana töpöttää ympäriinsä ja kuiskia itsekseen. Pomppu on erityisen kiinnostunut kaikesta uudesta ja jännittävästä.

**Lähtökohta:**

1.  **Arkinen Aamu:** Tarina alkaa tavallisesta aamusta Pompun kodissa. Aurinko paistaa, linnut visertävät, ja Pomppu on tyytyväinen elämäänsä. Mutta hän kaipaa pientä seikkailua.
2.  **Uteliaisuus Herää:** Pomppu tutkii taloa, kuten tavallista. Hän päätyy perunakellariin, johon hän ei ole koskaan aikaisemmin uskaltautunut. Perunakellarissa on hämärää ja hieman haisevaa, mutta nurkassa hän löytää jotain erikoista.
3.  **Salainen Ovi:** Perunakellarin takaseinässä, perunasäkkien takana, Pomppu löytää pienen, pyöreän oven. Ovi on koristeltu pienillä, kimaltavilla helmillä. Ovi on niin pieni, että Pompun on pon

### Generate the start of the story

In [107]:
starting_draft = generate_with_retry(model, starting_prompt.format(premise=premise, outline=outline)).text
pprint(starting_draft)

('Olipa kerran pörröinen marsu nimeltään Pomppu. Pomppu asui kodikkaassa '
 'talossa, jossa oli monta huonetta täynnä mielenkiintoisia paikkoja '
 'tutkittavaksi. Hän oli pieni, mutta hänen sydämensä oli täynnä suurta '
 'uteliaisuutta ja halua seikkailuun. Hänen turkkinsa oli pehmeää kuin '
 'untuvaa, ja hänen silmänsä olivat mustat kuin pienet nappipäättömyydet, '
 'jotka välkehtivät innosta aina kun hän huomasi jotain uutta. Pomppu rakasti '
 'töpöttää ympäriinsä, pienenä pallerona tutkien maailmaa pienin tassuin.\n'
 '\n'
 'Aamut Pompun talossa olivat aina erityisen ihania. Aurinko kipusi taivaalle, '
 'valaisten koko talon lämpimällä, kultaisella valollaan. Linnut lauloivat '
 'kirkkaita sävelmiään ikkunan takana, ja Pomppu heräsi jokaisena aamuna yhtä '
 'innoissaan. Hän käänsi itsensä makuulta, venytti pieniä varpaitaan ja '
 'töpötti nopeasti häkistään lattialle. Hän saattoi kuulla keittiöstä ääniä, '
 'jotka kertoivat aamupalasta, ja hänen vatsansa alkoi kurnia pienestä '
 'jä

### Generate the continuation of the story and check progress

In [108]:
draft=starting_draft
continuation = generate_with_retry(model, continuation_prompt.format(premise=premise, outline=outline, story_text=draft)).text
pprint(continuation)

('Pomppu ja naurahtavat porkkanatunnelit\n'
 '\n'
 'Pomppu katseli ympärilleen, hänen pieni sydämensä täynnä ihmetystä. Hän oli '
 'päässyt paikkaan, joka oli kuin suoraan unesta. Sateenkaaren väriset '
 'tunnelit olivat kuin loputtomat labyrintit, jotka johtivat aina uusiin '
 'ihmeisiin. Naurahtavat porkkanat olivat hänen parhaita ystäviään, ja yhdessä '
 'he olivat valmiit seikkailuun, mikä tahansa se olisikaan.\n'
 '\n'
 'Yhdessä tunnelin osiossa, joka oli aivan erityisen kimalteleva, Pomppu '
 'pysähtyi ihailemaan kimaltavia valopisteitä, jotka leijuivat ilmassa kuin '
 'pienet, tanssivat tähdet. Ne olivat aivan kuin taikavalot, jotka loivat '
 'ympärilleen ihmeellisen tunnelman. Pienet porkkanat pyörivät ja hyppelivät '
 'Pompun ympärillä, ja heidän kikatuksensa kaikui tunnelissa.\n'
 '\n'
 '"Tiedättekö te, mikä tämä paikka on?" Pomppu kysyi, äänessään innostunut '
 'sävy. Hän halusi tietää enemmän tästä uskomattomasta paikasta.\n'
 '\n'
 'Yksi porkkanoista, joka oli erityisen pu

### Let's finish writing the story.

After the iterative generation process is complete, the draft will contain the full story along with the `IAMDONE` signal at the end. The last part of this cell removes the `IAMDONE` signal and trims any unnecessary whitespace from the beginning and end of the draft, resulting in the final draft.

In [109]:
# Add the continuation to the initial draft, keep building the story until 'IAMDONE' is seen
draft = draft + '\n\n' + continuation

while 'IAMDONE' not in continuation:
  continuation = generate_with_retry(model, continuation_prompt.format(premise=premise, outline=outline, story_text=draft)).text
  draft = draft + '\n\n' + continuation

# Remove 'IAMDONE' and print the final story
final = draft.replace('IAMDONE', '').strip()
pprint(final)

('Olipa kerran pörröinen marsu nimeltään Pomppu. Pomppu asui kodikkaassa '
 'talossa, jossa oli monta huonetta täynnä mielenkiintoisia paikkoja '
 'tutkittavaksi. Hän oli pieni, mutta hänen sydämensä oli täynnä suurta '
 'uteliaisuutta ja halua seikkailuun. Hänen turkkinsa oli pehmeää kuin '
 'untuvaa, ja hänen silmänsä olivat mustat kuin pienet nappipäättömyydet, '
 'jotka välkehtivät innosta aina kun hän huomasi jotain uutta. Pomppu rakasti '
 'töpöttää ympäriinsä, pienenä pallerona tutkien maailmaa pienin tassuin.\n'
 '\n'
 'Aamut Pompun talossa olivat aina erityisen ihania. Aurinko kipusi taivaalle, '
 'valaisten koko talon lämpimällä, kultaisella valollaan. Linnut lauloivat '
 'kirkkaita sävelmiään ikkunan takana, ja Pomppu heräsi jokaisena aamuna yhtä '
 'innoissaan. Hän käänsi itsensä makuulta, venytti pieniä varpaitaan ja '
 'töpötti nopeasti häkistään lattialle. Hän saattoi kuulla keittiöstä ääniä, '
 'jotka kertoivat aamupalasta, ja hänen vatsansa alkoi kurnia pienestä '
 'jä

Language models like Gemini process text in units called tokens. For Gemini models, each token is equivalent to about 4 characters.

`Gemini-1.5-flash` has an output limit of 8192 tokens per generation call. This means that each individual prompt response cannot exceed this limit. By using iterative generation, you can create a story that is much longer than 8192 tokens by building it piece by piece.

Let's see how many tokens the final story is. Is it longer than 8192 tokens?

In [110]:
# Check the number of tokens in the final story
# gemini-1.5-flash output token limit is 8192
print(model.count_tokens(final))

total_tokens: 22611



## Next Steps

As an exercise, you can try to adjust the continuation prompt to take human-in-the-loop input to steer the narrative.